In [1]:
import glob
import os
import scipy.sparse as sps
import numpy as np
import pandas as pd

In [2]:
import random
import re
import os
import shutil
import sys
import json
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from collections import Counter


In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [5]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


In [6]:
# %pip install scikit-learn==0.23

In [6]:
from skmultilearn.problem_transform import LabelPowerset # pip3 install scikit-multilearn
from imblearn.over_sampling import RandomOverSampler

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
Dataset_ori = pd.read_csv('/Users/namrata/Documents/PSU/Thesis_research/Cybersecurity_framework/Data/OPP-115/data_to_gitlab/Cleaned_multilabel_dataset.txt',sep='\t')
del Dataset_ori['Other']
Dataset_ori.head(2)

,website,alltext,segment ID,Data Retention,Data Security,Do Not Track,First Party Collection/Use,International and Specific Audiences,Introductory/Generic,Policy Change,Practice not covered,Privacy contact information,Third Party Sharing/Collection,"User Access, Edit and Deletion",User Choice/Control
0,1017_sci-news.com,- details of your visits to our site including...,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1017_sci-news.com,"- if you contact us, we may keep a record of t...",2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# functions to preprocess words
import string
def preprocess(words):
    #we'll make use of python's translate function,that maps one set of characters to another
    #we create an empty mapping table, the third argument allows us to list all of the characters 
    #to remove during the translation process
    
    #first we will try to filter out some  unnecessary data like tabs
    table = str.maketrans('', '', '\t')
    table = str.maketrans('', '', '\n')
    words = [word.translate(table) for word in words]
    
    punctuations = (string.punctuation).replace("'", "") 
    # the character: ' appears in a lot of stopwords and changes meaning of words if removed
    #hence it is removed from the list of symbols that are to be discarded from the documents
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in words]
    
    #some white spaces may be added to the list of words, due to the translate function & nature of our documents
    #we remove them below
    words = [str for str in stripped_words if str]
    
    #some words are quoted in the documents & as we have not removed ' to maintain the integrity of some stopwords
    #we try to unquote such words below
    p_words = []
    for word in words:
        if (word[0] and word[len(word)-1] == "'"):
            word = word[1:len(word)-1]
        elif(word[0] == "'"):
            word = word[1:len(word)]
        else:
            word = word
        p_words.append(word)
    
    words = p_words.copy()
        
    #we will also remove just-numeric strings as they do not have any significant meaning in text classification
    words = [word for word in words if not word.isdigit()]
    
    #we will also remove single character strings
    words = [word for word in words if not len(word) == 1]
    
    #after removal of so many characters it may happen that some strings have become blank, we remove those
    words = [str for str in words if str]
    
    #we also normalize the cases of our words
    words = [word.lower() for word in words]
    
    #we try to remove words with only 2 characters
    words = [word for word in words if len(word) > 2]
    
    return words

In [10]:
StopWords = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th']


In [11]:
def remove_stopwords(words):
    words = [word for word in words if not word in StopWords]
    stop_words = list(set(stopwords.words('english')))
    words = [word for word in words if not word in stop_words]
    return words

def tokenize_sentence(words):
    words = preprocess(words)
    words = remove_stopwords(words)
    return words


In [12]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [13]:
def return_clean_words(text):
    text = clean_text(text)
    tokens = word_tokenize(text)
    clean_tokens =tokenize_sentence(tokens)
    return clean_tokens

In [14]:
return_clean_words(Dataset_ori['alltext'].iloc[0])

['details',
 'visits',
 'site',
 'including',
 'limited',
 'traffic',
 'data',
 'location',
 'data',
 'weblogs',
 'communication',
 'data']

### Train Word2Vec using all text data

In [15]:
import nltk
from nltk.stem import SnowballStemmer
import re
from gensim import utils

from gensim.models import Word2Vec
import gensim.models as g
import codecs

In [16]:
all_lists = []
for s in Dataset_ori['alltext']:
    all_lists.append(return_clean_words(s))


In [17]:
m = Word2Vec(all_lists, min_count=1,size= 300,workers=10, window =10, sg = 1)

In [18]:
Dataset_ori["Id"] = Dataset_ori['website'] + Dataset_ori['segment ID'].astype('str')
labels_order = Dataset_ori.columns[3:-1]
all_websites = Dataset_ori['website'].unique()
web_dict = Counter(all_websites)

selection_per_loop = dict()

fraction =1 

In [20]:
folds = 10

lp = LabelPowerset()
ros = RandomOverSampler(random_state=420)

err=0
for count in range(folds):
# count=0
    print('Fold: ',count)
    itemminValue = min(web_dict.items(), key=lambda x: x[1])

    min_val = itemminValue[1]
    minimum_webs = list()
    # print(min_val)
    for key, value in web_dict.items():
        if value == min_val:
            minimum_webs.append(key)
    random.shuffle(minimum_webs)

    selected_test_webs = minimum_webs[0:40]

    while len(selected_test_webs)<40:
        min_val+=1
        minimum_webs = list()
        for key, value in web_dict.items():
            if value == min_val:
                minimum_webs.append(key)
        random.shuffle(minimum_webs)
        selected_test_webs = list(set(selected_test_webs)|set(minimum_webs[0:40-len(selected_test_webs)]))
        # print('final len of test sites',len(selected_test_webs))

    for tw in selected_test_webs:
        web_dict[tw] += 1
    selection_per_loop[count] = selected_test_webs

    train_df = Dataset_ori[~Dataset_ori['website'].isin(selected_test_webs)]
    test_df = Dataset_ori[Dataset_ori['website'].isin(selected_test_webs)]

    train_df=train_df.sample(frac=fraction)
    test_df=test_df.sample(frac=fraction)

    train_text_ori = train_df['alltext'].tolist()
    # train_text_ori = [' '.join(t.split()) for t in train_text_ori]
    # train_text_ori = np.array(train_text_ori, dtype=object)[:, np.newaxis]
    train_label_ori = train_df.values[:,3:-1].astype(int)

    test_text = test_df['alltext'].tolist()
    # test_text = [' '.join(t.split()) for t in test_text]
    # test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    test_label = test_df.values[:,3:-1].astype(int)
    

    X1 = []
    for i in range(len(train_text_ori)):
        vec_inp = return_clean_words(train_text_ori[i])
        vector_S1 = []
        for s in vec_inp:
            try:
                vector_S1.append(m[s])
            except:
                continue
        vector1 = np.mean(vector_S1, axis=0,keepdims=True)
        
        if vector1.shape != (1,300):
            print('whole vec empty',err)
            err+=1
            vector1 = np.random.rand(1,300)
        X1.append(vector1)
    train_X = np.vstack(X1)

    X2 = []
    
    for i in range(len(test_text)):
        vec_inp = return_clean_words(test_text[i])
        vector_S1 = []
        for s in vec_inp:
            try:
                vector_S1.append(m[s])
            except:
                continue
        vector1 = np.mean(vector_S1, axis=0,keepdims=True)
        
        if vector1.shape != (1,300):
            print('whole vec empty',err)
            err+=1
            vector1 = np.random.rand(1,300)
        X2.append(vector1)
        
    test_X = np.vstack(X2)

    # print(test_X.shape)

    train_X_norm = min_max_scaler.fit_transform(train_X)
    test_X_norm = min_max_scaler.fit_transform(test_X)

    print('Before upsampling: ',train_X_norm.shape,train_label_ori.shape,test_X_norm.shape,test_label.shape)

    yt = lp.transform(train_label_ori)
    train_X_norm_up, y_resampled = ros.fit_sample(train_X_norm, yt)
    # train_text = train_text_ori
    # train_label = train_label_ori
    train_label = lp.inverse_transform(y_resampled).toarray()

    print('After Up-sampling',train_X_norm_up.shape,train_label.shape,test_X_norm.shape,test_label.shape)
    
    NB_pipeline = Pipeline([
                    #('tfidf', TfidfVectorizer(stop_words=stop_words)),
                    ('clf', OneVsRestClassifier(MultinomialNB(
                        fit_prior=True, class_prior=None))),
                ])        

    all_pred_res = []
    for category in range(train_label.shape[1]):
        NB_pipeline.fit(train_X_norm_up, train_label[:,category].reshape((train_label.shape[0], 1)))
        preds = NB_pipeline.predict(test_X_norm)
        all_pred_res.append(preds)

    # print(len(all_pred_res))
    pred_binary = np.asarray(all_pred_res).T

    # print(pred_binary.shape)

    result_scores = pd.DataFrame()
    avg_f1 = []
    for category in range(train_label.shape[1]):
        label_name = labels_order[category]

    #     lr_precision, lr_recall, _ = precision_recall_curve(test_label[:,category].astype('float'), pre_save_preds[:,category])
        lr_f1 = f1_score(test_label[:,category].astype(int), pred_binary[:,category])#, auc(lr_recall, lr_precision)

        if sum(test_label[:,category])==0 and sum(pred_binary[:,category])==0:
            tp=0
            fp=0
            tn=len(pred_binary[:,category])
            fn=0
        else:
            tn, fp, fn, tp = confusion_matrix(test_label[:,category].astype(int), pred_binary[:,category]).ravel()

        if tp+fp==0:
            precision = 0
        else:
            precision = float(tp)/float(tp+fp)

        if tp+fn==0:
            recall = 0
        else:
            recall = float(tp)/float(tp+fn)

        avg_f1.append(lr_f1)
        df = pd.DataFrame([{'Fold':count,'category':label_name,'tp':tp,'fp':fp,'tn':tn,'fn':fn,'recall':recall,'precision':precision,'f1':lr_f1}])

        result_scores = result_scores.append(df)
#     print(result_scores)
    print(np.mean(avg_f1))
    if count==0:
        result_scores.to_csv('.../Word2Vec_OPP_res_ori_upsampled_self_trained.csv',header=True,mode='a',sep='\t')
    else:
        result_scores.to_csv('.../Word2Vec_OPP_res_ori_upsampled_self_trained.csv',header=False,mode='a',sep='\t')

    count+=1 

Fold:  0
Before upsampling:  (2381, 300) (2381, 12) (1193, 300) (1193, 12)
After Up-sampling (45730, 300) (45730, 12) (1193, 300) (1193, 12)
0.40135494139286476
Fold:  1
Before upsampling:  (2269, 300) (2269, 12) (1305, 300) (1305, 12)
After Up-sampling (47964, 300) (47964, 12) (1305, 300) (1305, 12)
0.39507307542296655
Fold:  2
Before upsampling:  (2368, 300) (2368, 12) (1206, 300) (1206, 12)
After Up-sampling (45267, 300) (45267, 12) (1206, 300) (1206, 12)
0.42809798780414593
Fold:  3
Before upsampling:  (2307, 300) (2307, 12) (1267, 300) (1267, 12)
After Up-sampling (43497, 300) (43497, 12) (1267, 300) (1267, 12)
0.3988382345001267
Fold:  4
Before upsampling:  (2484, 300) (2484, 12) (1090, 300) (1090, 12)
After Up-sampling (55292, 300) (55292, 12) (1090, 300) (1090, 12)
0.3881493071191556
Fold:  5
Before upsampling:  (2365, 300) (2365, 12) (1209, 300) (1209, 12)
After Up-sampling (47120, 300) (47120, 12) (1209, 300) (1209, 12)
0.4221855514842421
Fold:  6
Before upsampling:  (2282, 3